In [1]:
import pandas as pd
import numpy as np


# 读取数据
train = pd.read_csv('train_smooth_all.csv')


In [2]:
train['date'] = pd.to_datetime(train['date'])
# 定义验证集的日期范围
val_start_date1 = pd.to_datetime('2023-06-03')
val_end_date1 = pd.to_datetime('2023-06-16')

val_start_date2 = pd.to_datetime('2024-05-03')
val_end_date2 = pd.to_datetime('2024-06-03')

# 划分第一个验证集
val1 = train[(train['date'] >= val_start_date1) & (train['date'] <= val_end_date1)]  # 2023-06-03 到 2023-06-16 的数据

# 划分第二个验证集
val2 = train[(train['date'] >= val_start_date2) & (train['date'] <= val_end_date2)]  # 2024-05-03 到 2024-06-03 的数据

# 合并两个验证集
val = pd.concat([val1, val2])

# 划分训练集
# 训练集是除了两个验证集之外的数据
train_before_val1 = train[train['date'] < val_start_date1]  # 第一个验证集之前的数据
train_between_vals = train[(train['date'] > val_end_date1) & (train['date'] < val_start_date2)]  # 两个验证集之间的数据
train_after_val2 = train[train['date'] > val_end_date2]  # 第二个验证集之后的数据

# 合并训练集
train = pd.concat([train_before_val1, train_between_vals, train_after_val2])

# 检查数据量
print("训练集数据量：", len(train))
print("验证集数据量：", len(val))

训练集数据量： 3853808
验证集数据量： 153559


In [3]:
train_date = train['date']
val_date = val['date']
train.drop(['unique_id','date'], axis=1, inplace=True)
val.drop(['unique_id','date'], axis=1, inplace=True)
y_train = train['sales']
y_val = val['sales']
train = train.drop(columns=['sales'])
val = val.drop(columns=['sales'])
y_train

1          12.63
2          34.55
3          34.52
4          35.92
6           6.39
           ...  
4007362    26.56
4007363    27.42
4007364    33.39
4007365    22.88
4007366    32.10
Name: sales, Length: 3853808, dtype: float64

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 定义随机森林回归模型
rf_regressor = RandomForestRegressor(
    n_estimators=100,       # 树的数量
    criterion='squared_error',  # 分裂标准
    max_depth=None,         # 树的最大深度
    min_samples_split=2,    # 分裂内部节点所需的最小样本数
    min_samples_leaf=1,     # 叶节点所需的最小样本数
    max_leaf_nodes=None,    # 最大叶节点数量
    min_impurity_decrease = 0.0,  # 分裂节点的最小不纯度减少量
    # max_features='auto',    # 寻找最佳分裂时考虑的特征数量
    bootstrap=True,         # 是否使用 bootstrap 采样
    random_state=42,         # 随机种子
    n_jobs=-1,               # 使用所有可用的 CPU 核心
    verbose=2                # 输出调优过程
)

In [5]:
# 训练模型
rf_regressor.fit(train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min


building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100


MemoryError: could not allocate 268435456 bytes

In [ ]:
# 使用训练好的模型进行预测
y_pred = rf_regressor.predict(val)

mae = mean_absolute_error(y_val, y_pred)
print(f'MAE: {mae}')

# 计算均方误差 (MSE)
mse = mean_squared_error(y_val, y_pred)
print(f'MSE: {mse}')

# 计算 R² (决定系数)
r2 = r2_score(y_val, y_pred)
print(f'R² Score: {r2}')

# 输出预测结果
print('Predictions:', y_pred)




In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# 计算分位数
bins = np.quantile(y_val, [0.33, 0.66])

# 将目标变量分为低、中、高三个区间
y_val_binned = np.digitize(y_val, bins)

# 输出区间范围
print(f'Low range: y_val < {bins[0]:.2f}')
print(f'Mid range: {bins[0]:.2f} <= y_val < {bins[1]:.2f}')
print(f'High range: y_val >= {bins[1]:.2f}')

# 计算每个区间的 MAE
for i in range(3):
    mask = y_val_binned == i
    mae = mean_absolute_error(y_val[mask], y_pred[mask])
    print(f'MAE for bin {i}: {mae:.2f}')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 定义验证集的日期范围
val_start_date1 = pd.to_datetime('2023-06-03')
val_end_date1 = pd.to_datetime('2023-06-16')

val_start_date2 = pd.to_datetime('2024-05-03')
val_end_date2 = pd.to_datetime('2024-06-03')

train['date'] = train_date
val['date'] = val_date

train['sales'] = y_train
val['sales'] = y_val
# 确保 train 和 val 的 date 列是 datetime 类型
train['date'] = pd.to_datetime(train['date'])
val['date'] = pd.to_datetime(val['date'])

# 将预测值添加到验证集中
val['sales_hat'] = y_pred

# 划分训练集为多个部分
train_before_val1 = train[train['date'] < val_start_date1]  # 第一个验证集之前的数据
train_between_vals = train[(train['date'] > val_end_date1) & (train['date'] < val_start_date2)]  # 两个验证集之间的数据
train_after_val2 = train[train['date'] > val_end_date2]  # 第二个验证集之后的数据

# 划分验证集为多个部分
val1 = val[(val['date'] >= val_start_date1) & (val['date'] <= val_end_date1)]  # 第一个验证集
val2 = val[(val['date'] >= val_start_date2) & (val['date'] <= val_end_date2)]  # 第二个验证集

# 仅保留 2023 年和 2024 年的数据
train_before_val1 = train_before_val1[train_before_val1['date'].dt.year.isin([2023, 2024])]
train_between_vals = train_between_vals[train_between_vals['date'].dt.year.isin([2023, 2024])]
train_after_val2 = train_after_val2[train_after_val2['date'].dt.year.isin([2023, 2024])]
val1 = val1[val1['date'].dt.year.isin([2023, 2024])]
val2 = val2[val2['date'].dt.year.isin([2023, 2024])]

# 绘制图形
plt.figure(figsize=(15, 8))

# 绘制第一个验证集之前训练集的实际销售数据（蓝色）
sns.lineplot(data=train_before_val1, x='date', y='sales', color='blue', label='Train Sales ')

# 绘制两个验证集之间训练集的实际销售数据（蓝色）
sns.lineplot(data=train_between_vals, x='date', y='sales', color='blue')

# 绘制第二个验证集之后训练集的实际销售数据（蓝色）
sns.lineplot(data=train_after_val2, x='date', y='sales', color='blue')

# 绘制第一个验证集的实际销售数据（黄色）
sns.lineplot(data=val1, x='date', y='sales', color='yellow', label='Validation True Sales')

# 绘制第二个验证集的实际销售数据（黄色）
sns.lineplot(data=val2, x='date', y='sales', color='yellow')

# 绘制第一个验证集的预测销售数据（红色虚线）
sns.lineplot(data=val1, x='date', y='sales_hat', color='red', linestyle='--', label='Predicted Sales')

# 绘制第二个验证集的预测销售数据（红色虚线）
sns.lineplot(data=val2, x='date', y='sales_hat', color='red', linestyle='--')

# 添加标题和标签
plt.title('Sales and Sales Predictions Over Time (2023-2024)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 绘制图形
plt.figure(figsize=(15, 8))

# 绘制第一个验证集的实际销售数据（黄色）
sns.lineplot(data=val1, x='date', y='sales', color='yellow', label='Validation True Sales')

# 绘制第一个验证集的预测销售数据（红色虚线）
sns.lineplot(data=val1, x='date', y='sales_hat', color='red', linestyle='--', label='Predicted Sales')

# 添加标题和标签
plt.title('Validation Set: True vs Predicted Sales (2023-2024)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# 绘制图形
plt.figure(figsize=(15, 8))

# 绘制第二个验证集的实际销售数据（黄色）
sns.lineplot(data=val2, x='date', y='sales', color='yellow', label='Validation True Sales')

# 绘制第二个验证集的预测销售数据（红色虚线）
sns.lineplot(data=val2, x='date', y='sales_hat', color='red', linestyle='--', label='Predicted Sales')

# 添加标题和标签
plt.title('Validation Set: True vs Predicted Sales (2023-2024)')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid()
plt.show()